# 生成所需数据

In [1]:
import os
import pandas as pd
import numpy as np
import shutil
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *

In [2]:
# 指定工作空间
env.workspace = r'C:\Users\Runker\Desktop\ele_sb'
env.overwriteOutput = True

In [3]:
# 三调地块路径
sd_data = r'F:\cache_data\shp_file\sb\sb_sd_dltb.shp'
# 母质路径
mz_data = r'F:\cache_data\shp_file\sb\sb_dz_river_sd_extent.shp'
# 坡位矢量路径
slope_data = r'F:\cache_data\shp_file\sb\sb_slopeclass_smooth.shp'

In [4]:
# 提取耕林园草
arcpy.Select_analysis(sd_data,"sb_sd_dlyc.shp",f"{'DLDM'} IN ('01', '02', '03','04','12')")

<Result 'C:\\Users\\Runker\\Desktop\\ele_sb\\sb_sd_dlyc.shp'>

In [6]:
# 融合母质
arcpy.Dissolve_management(in_features=mz_data,out_feature_class='sb_dz_result_dissove.shp',dissolve_field='改母岩',multi_part='SINGLE_PART')

<Result 'C:\\Users\\Runker\\Desktop\\ele_sb\\sb_dz_result_dissove.shp'>

In [7]:
# 相交
arcpy.Intersect_analysis(in_features=['sb_sd_dlyc','sb_dz_result_dissove',slope_data],out_feature_class='sb_merge_data',join_attributes='ALL')    

<Result 'C:\\Users\\Runker\\Desktop\\ele_sb\\sb_merge_data.shp'>

In [8]:
# 转单部件
arcpy.MultipartToSinglepart_management('sb_merge_data','sb_merge_data_single')


<Result 'C:\\Users\\Runker\\Desktop\\ele_sb\\sb_merge_data_single.shp'>

In [4]:
import arcpy
import traceback

def merge_small_parcels(input_fc, output_fc, land_type_field, dz_field, thresholds):
    try:
        # 复制输入要素类
        arcpy.CopyFeatures_management(input_fc, output_fc)
        print(f"已创建输出要素类: {output_fc}")

        # 确保存在面积字段
        if "Area" not in [f.name for f in arcpy.ListFields(output_fc)]:
            arcpy.AddField_management(output_fc, "Area", "DOUBLE")
            print("已添加 Area 字段")

        # 计算面积
        arcpy.CalculateField_management(output_fc, "Area", "!shape.area!", "PYTHON3")
        print("已计算面积")

        # 创建图层
        arcpy.MakeFeatureLayer_management(output_fc, "output_layer")

        # 获取小面积图斑
        small_parcels = []
        with arcpy.da.SearchCursor(output_fc, ["OID@", land_type_field, dz_field, "Area", "SHAPE@"]) as cursor:
            for row in cursor:
                oid, land_type, dz, area, shape = row
                threshold = thresholds.get(dz, min(thresholds.values()))  # 如果没有指定阈值，使用最小的阈值
                if area < threshold:
                    small_parcels.append((oid, land_type, dz, area, shape))

        print(f"找到 {len(small_parcels)} 个小于指定阈值的图斑")

        # 处理小面积图斑
        merged_count = 0
        with arcpy.da.Editor(arcpy.Describe(output_fc).path) as editor:
            for small_parcel in small_parcels:
                oid, land_type, dz, area, shape = small_parcel
                
                # 选择相邻图斑
                arcpy.SelectLayerByLocation_management("output_layer", "BOUNDARY_TOUCHES", shape)
                
                # 在选择集中查找最大的相同地类图斑
                max_area = 0
                max_oid = None
                with arcpy.da.SearchCursor("output_layer", ["OID@", land_type_field, "Area"]) as neighbor_cursor:
                    for neighbor in neighbor_cursor:
                        if neighbor[0] != oid and neighbor[1] == land_type and neighbor[2] > max_area:
                            max_area = neighbor[2]
                            max_oid = neighbor[0]
                
                if max_oid:
                    try:
                        # 合并几何
                        where_clause = f"OBJECTID IN ({oid}, {max_oid})"
                        with arcpy.da.UpdateCursor(output_fc, ["OID@", "SHAPE@", land_type_field], where_clause) as update_cursor:
                            shapes = []
                            for update_row in update_cursor:
                                shapes.append(update_row[1])
                                if update_row[0] == oid:
                                    update_cursor.deleteRow()
                            
                            if len(shapes) == 2:
                                merged_shape = shapes[0].union(shapes[1])
                                update_cursor.reset()
                                for update_row in update_cursor:
                                    if update_row[0] == max_oid:
                                        update_cursor.updateRow([max_oid, merged_shape, land_type])
                                        merged_count += 1
                                        break
                    except Exception as e:
                        print(f"处理 OID {oid} 时出错: {str(e)}")
                        print(traceback.format_exc())
                
                # 清除选择
                arcpy.SelectLayerByAttribute_management("output_layer", "CLEAR_SELECTION")

                if merged_count % 100 == 0:
                    print(f"已处理 {merged_count} 个小面积图斑")

        # 重新计算面积
        arcpy.CalculateField_management(output_fc, "Area", "!shape.area!", "PYTHON3")
        print(f"操作完成。合并了 {merged_count} 个小面积图斑到相邻的相同地类图斑中。")
        print(f"结果保存在新的要素类: {output_fc}")

    except Exception as e:
        print(f"发生错误: {str(e)}")
        print(traceback.format_exc())



In [5]:
# 使用示例
if __name__ == "__main__":
    input_fc = "DY_SD_MZ_SLOPEPOSITION_INTERSECT_SINGLE_ELIMITE"
    output_fc = "DY_SD_MZ_SLOPEPOSITION_INTERSECT_SINGLE_ELIMITE_2"
    land_type_field = "DLMC"
    dz_field = "DZ"
    thresholds = {
        "01": 50,
        "03": 1000,
        "04": 1000,
        # 可以继续添加其他DZ值的阈值
    }

    merge_small_parcels(input_fc, output_fc, land_type_field, dz_field, thresholds)

已创建输出要素类: DY_SD_MZ_SLOPEPOSITION_INTERSECT_SINGLE_ELIMITE_2
已计算面积
找到 6627 个小于指定阈值的图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0 个小面积图斑
已处理 0

In [7]:
import traceback

# 输入和输出要素类
input_fc = r"D:\ArcGISProjects\workspace\shbyq\DZ.gdb\DY_SD_MZ_SLOPEPOSITION_INTERSECT_SINGLE"
output_fc = "select_train_calc_end"
# 地类字段名称
land_type_field = "DLMC"

# 面积阈值（平方米）
area_threshold = 50

try:
    # 复制输入要素类
    arcpy.CopyFeatures_management(input_fc, output_fc)
    print(f"已创建输出要素类: {output_fc}")

    # 确保存在面积字段
    if "Area" not in [f.name for f in arcpy.ListFields(output_fc)]:
        arcpy.AddField_management(output_fc, "Area", "DOUBLE")
        print("已添加 Area 字段")

    # 计算面积
    arcpy.CalculateField_management(output_fc, "Area", "!shape.area!", "PYTHON3")
    print("已计算面积")

    # 创建图层
    arcpy.MakeFeatureLayer_management(output_fc, "output_layer")

    # 获取小面积图斑
    small_parcels = []
    with arcpy.da.SearchCursor(output_fc, ["OID@", land_type_field, "Area", "SHAPE@"]) as cursor:
        for row in cursor:
            if row[2] < area_threshold:
                small_parcels.append(row)

    print(f"找到 {len(small_parcels)} 个小于 {area_threshold} 平方米的图斑")

    # 处理小面积图斑
    merged_count = 0
    with arcpy.da.Editor(arcpy.env.workspace) as editor:
        for small_parcel in small_parcels:
            oid, land_type, area, shape = small_parcel
            
            # 选择相邻图斑
            arcpy.SelectLayerByLocation_management("output_layer", "BOUNDARY_TOUCHES", shape)
            
            # 在选择集中查找最大的相同地类图斑
            max_area = 0
            max_oid = None
            with arcpy.da.SearchCursor("output_layer", ["OID@", land_type_field, "Area"]) as neighbor_cursor:
                for neighbor in neighbor_cursor:
                    if neighbor[0] != oid and neighbor[1] == land_type and neighbor[2] > max_area:
                        max_area = neighbor[2]
                        max_oid = neighbor[0]
            
            if max_oid:
                try:
                    # 合并几何
                    where_clause = f"OBJECTID IN ({oid}, {max_oid})"
                    with arcpy.da.UpdateCursor(output_fc, ["OID@", "SHAPE@", land_type_field], where_clause) as update_cursor:
                        shapes = []
                        for update_row in update_cursor:
                            shapes.append(update_row[1])
                            if update_row[0] == oid:
                                update_cursor.deleteRow()
                        
                        if len(shapes) == 2:
                            merged_shape = shapes[0].union(shapes[1])
                            update_cursor.reset()
                            for update_row in update_cursor:
                                if update_row[0] == max_oid:
                                    update_cursor.updateRow([max_oid, merged_shape, land_type])
                                    merged_count += 1
                                    break
                except Exception as e:
                    print(f"处理 OID {oid} 时出错: {str(e)}")
                    print(traceback.format_exc())
            
            # 清除选择
            arcpy.SelectLayerByAttribute_management("output_layer", "CLEAR_SELECTION")

            if merged_count % 100 == 0:
                print(f"已处理 {merged_count} 个小面积图斑")

    # 重新计算面积
    arcpy.CalculateField_management(output_fc, "Area", "!shape.area!", "PYTHON3")
    print(f"操作完成。合并了 {merged_count} 个小面积图斑到相邻的相同地类图斑中。")
    print(f"结果保存在新的要素类: {output_fc}")

except Exception as e:
    print(f"发生错误: {str(e)}")
    print(traceback.format_exc())

已创建输出要素类: select_train_calc_end
已计算面积
找到 2181 个小于 50 平方米的图斑
已处理 100 个小面积图斑
已处理 200 个小面积图斑
已处理 300 个小面积图斑
已处理 400 个小面积图斑
已处理 500 个小面积图斑
已处理 600 个小面积图斑
已处理 700 个小面积图斑
已处理 800 个小面积图斑
已处理 900 个小面积图斑
已处理 1000 个小面积图斑
已处理 1100 个小面积图斑
已处理 1200 个小面积图斑
已处理 1300 个小面积图斑
已处理 1400 个小面积图斑
已处理 1500 个小面积图斑
已处理 1600 个小面积图斑
已处理 1700 个小面积图斑
已处理 1800 个小面积图斑
已处理 1900 个小面积图斑
已处理 2000 个小面积图斑
已处理 2100 个小面积图斑
操作完成。合并了 2175 个小面积图斑到相邻的相同地类图斑中。
结果保存在新的要素类: select_train_calc_end


In [8]:
# 坡位栅格处理
# 输入栅格
input_raster = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb\SlopeClass"

# 输出文件
reclass_raster = "reclass_raster"
vector_output = "vector_output"
final_output = "reclass_slope"

# 面积阈值（平方米）
area_threshold = 100

try:
    # 步骤1：重分类栅格
    print("开始栅格重分类...")
    remap = arcpy.sa.RemapRange([[1, 2, 1], [3, 4, 2], [5, 6, 3]])
    reclass = arcpy.sa.Reclassify(input_raster, "VALUE", remap, "NODATA")
    reclass.save(reclass_raster)
    print("栅格重分类完成")

    # 步骤2：栅格转面
    print("开始栅格转面...")
    arcpy.RasterToPolygon_conversion(reclass_raster, vector_output, "SIMPLIFY")
    print("栅格转面完成")

    # 步骤3：添加并计算面积字段
    print("计算面积...")
    arcpy.AddField_management(vector_output, "Area_sqm", "DOUBLE")
    arcpy.CalculateField_management(vector_output, "Area_sqm", "!shape.area!", "PYTHON3")

    # 检查字段是否成功添加
    fields = [f.name for f in arcpy.ListFields(vector_output)]
    if "Area_sqm" not in fields:
        raise ValueError("面积字段 'Area_sqm' 未成功添加")
    # 步骤4：消除小面积图斑
    print(f"开始消除面积小于 {area_threshold} 平方米的图斑...")
    
    # 创建图层
    arcpy.MakeFeatureLayer_management(vector_output, "vector_layer")
    
    # 选择小面积图斑
    selection = arcpy.SelectLayerByAttribute_management("vector_layer", "NEW_SELECTION", f"Area_sqm < {area_threshold}")
    
    # 检查选择结果
    result = arcpy.GetCount_management(selection)
    count = int(result.getOutput(0))
    print(f"选择了 {count} 个小面积图斑")

    if count > 0:
        # 使用Eliminate工具消除小面积图斑
        arcpy.Eliminate_management("vector_layer", final_output, "AREA")
        print("小面积图斑消除完成")
    else:
        print("没有找到小面积图斑，复制原始图层作为最终输出")
        arcpy.CopyFeatures_management(vector_output, final_output)

    # 清理中间数据
    arcpy.Delete_management(reclass_raster)
    arcpy.Delete_management(vector_output)

    print(f"处理完成。最终结果保存在：{final_output}")

except arcpy.ExecuteError:
    print("ArcPy错误:")
    print(arcpy.GetMessages(2))
except Exception as e:
    print(f"发生错误: {str(e)}")
    import traceback
    print(traceback.format_exc())


开始栅格重分类...
栅格重分类完成
开始栅格转面...
栅格转面完成
计算面积...
开始消除面积小于 100 平方米的图斑...
选择了 7 个小面积图斑
小面积图斑消除完成
处理完成。最终结果保存在：reclass_slope


In [14]:
# 和坡位相交
# 输入要素类
input_A = "DY_SD_CLASS_INTERSECT_reclass"
input_B = "reclass_slope"


# 临时和输出要素类
temp_selected_A = "in_memory\\Selected_A"
temp_intersect = "in_memory\\Intersect_Result"
temp_output_A = "in_memory\\Temp_Output_A"
output_A = "DY_SD_CLASS_INTERSECT_reclass_slope_single"

# DZ字段名称
dz_field = "DZ"

try:
    # 步骤1：选择A中DZ字段为03和04的要素
    print("正在选择A中DZ字段为03和04的要素...")
    arcpy.MakeFeatureLayer_management(input_A, "A_layer")
    arcpy.SelectLayerByAttribute_management("A_layer", "NEW_SELECTION", f"{dz_field} IN ('03', '04')")
    arcpy.CopyFeatures_management("A_layer", temp_selected_A)
    
    selected_count = int(arcpy.GetCount_management(temp_selected_A)[0])
    print(f"选择了 {selected_count} 个要素")

    # 步骤2：对选中的要素与B进行相交操作，保留所有属性字段
    print("正在进行相交操作...")
    arcpy.Intersect_analysis([temp_selected_A, input_B], temp_intersect, "ALL")

    # 步骤3：精确更新A要素类
    print("正在更新A要素类...")
    
    # 创建一个新的要素类，包含A的所有要素
    arcpy.CopyFeatures_management(input_A, temp_output_A)
    
    # 使用空间连接找出需要被替换的区域
    temp_joined = "in_memory\\Temp_Joined"
    arcpy.SpatialJoin_analysis(temp_output_A, temp_intersect, temp_joined, 
                               "JOIN_ONE_TO_ONE", "KEEP_ALL", 
                               match_option="INTERSECT")
    
    # 删除那些在原A中DZ为03或04且与B相交的区域
    with arcpy.da.UpdateCursor(temp_joined, [dz_field, "Join_Count"]) as cursor:
        for row in cursor:
            if row[0] in ['03', '04'] and row[1] > 0:
                cursor.deleteRow()

    # 将相交结果追加到更新后的A
    print("追加相交结果...")
    arcpy.Append_management(temp_intersect, temp_joined, "NO_TEST")

    # 步骤4：将多部键转换为单部键
    print("正在将多部键转换为单部键...")
    arcpy.MultipartToSinglepart_management(temp_joined, output_A)

    # 计算处理后的要素数
    final_count = int(arcpy.GetCount_management(output_A)[0])
    print(f"处理完成。更新后的A要素类（单部键，包含B的属性）保存为：{output_A}")
    print(f"最终要素数：{final_count}")

    # 清理临时数据
    print("清理临时数据...")
    for temp_fc in [temp_selected_A, temp_intersect, temp_output_A, temp_joined]:
        if arcpy.Exists(temp_fc):
            arcpy.Delete_management(temp_fc)

except arcpy.ExecuteError:
    print("ArcPy错误:")
    print(arcpy.GetMessages(2))
except Exception as e:
    print(f"发生错误: {str(e)}")
    import traceback
    print(traceback.format_exc())

正在选择A中DZ字段为03和04的要素...
选择了 45575 个要素
正在进行相交操作...
正在更新A要素类...
追加相交结果...
正在将多部键转换为单部键...
处理完成。更新后的A要素类（单部键，包含B的属性）保存为：DY_SD_CLASS_INTERSECT_reclass_slope_single
最终要素数：220838
清理临时数据...


In [6]:
# 自然土和坡位
# 输入要素类
input_A = "DY_SD_CLASS_INTERSECT_reclass"
input_B = "reclass_slope"

# 临时和输出要素类
temp_selected_A = "in_memory\\Selected_A"
temp_intersect = "in_memory\\Intersect_Result"
output_fc = "DY_SD_CLASS_INTERSECT_reclass_slope_single"

# 字段名称
dz_field = "DZ"
dlmc_field = "DLMC"
gridcode_field = "gridcode"
area_field = "Area"

# 面积阈值
area_threshold = 1000

def add_field_if_not_exists(fc, field_name, field_type):
    if field_name not in [f.name for f in arcpy.ListFields(fc)]:
        arcpy.AddField_management(fc, field_name, field_type)
        print(f"字段 {field_name} 已添加")

try:
    # 步骤1：选择A中DZ字段为03和04的要素
    print("正在选择A中DZ字段为03和04的要素...")
    arcpy.MakeFeatureLayer_management(input_A, "A_layer")
    arcpy.SelectLayerByAttribute_management("A_layer", "NEW_SELECTION", f"{dz_field} IN ('03', '04')")
    arcpy.CopyFeatures_management("A_layer", temp_selected_A)
    
    selected_count = int(arcpy.GetCount_management(temp_selected_A)[0])
    print(f"选择了 {selected_count} 个要素")

    # 步骤2：对选中的要素与B进行相交操作
    print("正在进行相交操作...")
    arcpy.Intersect_analysis([temp_selected_A, input_B], temp_intersect, "ALL")

    # 步骤3：保留A的所有字段和B的gridcode字段
    print("调整字段...")
    a_fields = [f.name for f in arcpy.ListFields(input_A) if f.name not in ['OBJECTID', 'Shape', 'Shape_Length', 'Shape_Area']]
    fields_to_keep = a_fields + [gridcode_field]
    all_fields = [f.name for f in arcpy.ListFields(temp_intersect)]
    fields_to_delete = [f for f in all_fields if f not in fields_to_keep and f not in ['OBJECTID', 'Shape', 'Shape_Length', 'Shape_Area']]
    if fields_to_delete:
        arcpy.DeleteField_management(temp_intersect, fields_to_delete)

    # 步骤4：处理小面积图斑
    print("正在处理小面积图斑...")
    add_field_if_not_exists(temp_intersect, area_field, "DOUBLE")
    arcpy.CalculateField_management(temp_intersect, area_field, "!shape.area!", "PYTHON3")

    arcpy.MakeFeatureLayer_management(temp_intersect, "intersect_layer")

    # 获取小面积图斑
    small_parcels = []
    with arcpy.da.SearchCursor(temp_intersect, ["OID@", dlmc_field, dz_field, area_field, "SHAPE@"]) as cursor:
        for row in cursor:
            if row[2] in ['03', '04'] and row[3] < area_threshold:
                small_parcels.append(row)

    print(f"找到 {len(small_parcels)} 个小于 {area_threshold} 平方米的图斑")

    # 处理小面积图斑
    merged_count = 0
    with arcpy.da.Editor(arcpy.env.workspace) as editor:
        for small_parcel in small_parcels:
            oid, land_type, dz, area, shape = small_parcel
            
            # 选择相邻图斑
            arcpy.SelectLayerByLocation_management("intersect_layer", "BOUNDARY_TOUCHES", shape)
            
            # 在选择集中查找最大的相同地类和DZ的图斑
            max_area = 0
            max_oid = None
            with arcpy.da.SearchCursor("intersect_layer", ["OID@", dlmc_field, dz_field, area_field]) as neighbor_cursor:
                for neighbor in neighbor_cursor:
                    if neighbor[0] != oid and neighbor[1] == land_type and neighbor[2] == dz and neighbor[3] > max_area:
                        max_area = neighbor[3]
                        max_oid = neighbor[0]
            
            if max_oid:
                try:
                    # 合并几何
                    where_clause = f"OBJECTID IN ({oid}, {max_oid})"
                    with arcpy.da.UpdateCursor(temp_intersect, ["OID@", "SHAPE@", dlmc_field, dz_field, gridcode_field], where_clause) as update_cursor:
                        shapes = []
                        gridcode = None
                        for update_row in update_cursor:
                            shapes.append(update_row[1])
                            if update_row[0] == max_oid:
                                gridcode = update_row[4]
                            if update_row[0] == oid:
                                update_cursor.deleteRow()
                        
                        if len(shapes) == 2:
                            merged_shape = shapes[0].union(shapes[1])
                            update_cursor.reset()
                            for update_row in update_cursor:
                                if update_row[0] == max_oid:
                                    update_cursor.updateRow([max_oid, merged_shape, land_type, dz, gridcode])
                                    merged_count += 1
                                    break
                except Exception as e:
                    print(f"处理 OID {oid} 时出错: {str(e)}")
                    print(traceback.format_exc())
            
            # 清除选择
            arcpy.SelectLayerByAttribute_management("intersect_layer", "CLEAR_SELECTION")

            if merged_count % 100 == 0 and merged_count > 0:
                print(f"已处理 {merged_count} 个小面积图斑")

    # 步骤5：更新A要素类
    print("正在更新A要素类...")
    arcpy.CopyFeatures_management(input_A, output_fc)
    
    # 删除那些在原A中DZ为03或04的区域
    arcpy.MakeFeatureLayer_management(output_fc, "output_layer")
    arcpy.SelectLayerByAttribute_management("output_layer", "NEW_SELECTION", f"{dz_field} IN ('03', '04')")
    arcpy.DeleteFeatures_management("output_layer")

    # 将处理后的结果追加到更新后的A
    print("追加处理后的结果...")
    arcpy.Append_management(temp_intersect, output_fc, "NO_TEST")

    # 步骤6：将多部键转换为单部键
    print("正在将多部键转换为单部键...")
    arcpy.MultipartToSinglepart_management(output_fc, output_fc + "_single")

    # 计算处理后的要素数
    final_count = int(arcpy.GetCount_management(output_fc + "_single")[0])
    print(f"处理完成。更新后的A要素类（单部键，包含A的所有字段和B的gridcode字段，小面积已处理）保存为：{output_fc}_single")
    print(f"最终要素数：{final_count}")

    # 清理临时数据
    print("清理临时数据...")
    for temp_fc in [temp_selected_A, temp_intersect]:
        if arcpy.Exists(temp_fc):
            arcpy.Delete_management(temp_fc)

except arcpy.ExecuteError:
    print("ArcPy错误:")
    print(arcpy.GetMessages(2))
except Exception as e:
    print(f"发生错误: {str(e)}")
    print(traceback.format_exc())

已处理 9900 个小面积图斑
已处理 10000 个小面积图斑
已处理 10100 个小面积图斑
已处理 10100 个小面积图斑
已处理 10200 个小面积图斑
已处理 10200 个小面积图斑
已处理 10200 个小面积图斑
已处理 10200 个小面积图斑
已处理 10300 个小面积图斑
已处理 10300 个小面积图斑
已处理 10400 个小面积图斑
已处理 10500 个小面积图斑
已处理 10600 个小面积图斑
已处理 10700 个小面积图斑
已处理 10700 个小面积图斑
已处理 10800 个小面积图斑
已处理 10900 个小面积图斑
已处理 11000 个小面积图斑
已处理 11000 个小面积图斑
已处理 11100 个小面积图斑
已处理 11200 个小面积图斑
已处理 11200 个小面积图斑
已处理 11300 个小面积图斑
已处理 11400 个小面积图斑
已处理 11500 个小面积图斑
已处理 11600 个小面积图斑
已处理 11600 个小面积图斑
已处理 11700 个小面积图斑
已处理 11800 个小面积图斑
已处理 11900 个小面积图斑
已处理 11900 个小面积图斑
已处理 12000 个小面积图斑
已处理 12100 个小面积图斑
已处理 12100 个小面积图斑
已处理 12200 个小面积图斑
已处理 12300 个小面积图斑
已处理 12400 个小面积图斑
已处理 12400 个小面积图斑
已处理 12500 个小面积图斑
已处理 12600 个小面积图斑
已处理 12600 个小面积图斑
已处理 12700 个小面积图斑
已处理 12800 个小面积图斑
已处理 12900 个小面积图斑
已处理 13000 个小面积图斑
已处理 13000 个小面积图斑
已处理 13000 个小面积图斑
已处理 13100 个小面积图斑
已处理 13100 个小面积图斑
已处理 13200 个小面积图斑
已处理 13300 个小面积图斑
已处理 13400 个小面积图斑
已处理 13400 个小面积图斑
已处理 13400 个小面积图斑
已处理 13500 个小面积图斑
已处理 13500 个小面积图斑
已处理 13500 个小面积图斑
已处理 13600 个小面积图斑
已处理 13700 个小面积图

In [ ]:
# 输入要素类
input_A = "DY_SD_CLASS_INTERSECT_reclass"
input_B = "reclass_slope"

# 临时和输出要素类
temp_selected_A = "in_memory\\Selected_A"
temp_intersect = "in_memory\\Intersect_Result"
temp_singlepart = "in_memory\\Singlepart_Result"
output_fc = "DY_SD_CLASS_INTERSECT_reclass_slope_single"

# 字段名称
dz_field = "DZ"
dlmc_field = "DLMC"
gridcode_field = "gridcode"
area_field = "Area"

# 面积阈值
area_threshold = 1000

def add_field_if_not_exists(fc, field_name, field_type):
    if field_name not in [f.name for f in arcpy.ListFields(fc)]:
        arcpy.AddField_management(fc, field_name, field_type)
        print(f"字段 {field_name} 已添加")

try:
    # 步骤1：选择A中DZ字段为03和04的要素
    print("正在选择A中DZ字段为03和04的要素...")
    arcpy.MakeFeatureLayer_management(input_A, "A_layer")
    arcpy.SelectLayerByAttribute_management("A_layer", "NEW_SELECTION", f"{dz_field} IN ('03', '04')")
    arcpy.CopyFeatures_management("A_layer", temp_selected_A)
    
    selected_count = int(arcpy.GetCount_management(temp_selected_A)[0])
    print(f"选择了 {selected_count} 个要素")

    # 步骤2：对选中的要素与B进行相交操作
    print("正在进行相交操作...")
    arcpy.Intersect_analysis([temp_selected_A, input_B], temp_intersect, "ALL")

    # 步骤3：保留A的所有字段和B的gridcode字段
    print("调整字段...")
    a_fields = [f.name for f in arcpy.ListFields(input_A) if f.name not in ['OBJECTID', 'Shape', 'Shape_Length', 'Shape_Area']]
    fields_to_keep = a_fields + [gridcode_field]
    all_fields = [f.name for f in arcpy.ListFields(temp_intersect)]
    fields_to_delete = [f for f in all_fields if f not in fields_to_keep and f not in ['OBJECTID', 'Shape', 'Shape_Length', 'Shape_Area']]
    if fields_to_delete:
        arcpy.DeleteField_management(temp_intersect, fields_to_delete)

    # 步骤4：将多部键转换为单部键
    print("正在将多部键转换为单部键...")
    arcpy.MultipartToSinglepart_management(temp_intersect, temp_singlepart)

    # 步骤5：处理小面积图斑
    print("正在处理小面积图斑...")
    add_field_if_not_exists(temp_singlepart, area_field, "DOUBLE")
    arcpy.CalculateField_management(temp_singlepart, area_field, "!shape.area!", "PYTHON3")

    arcpy.MakeFeatureLayer_management(temp_singlepart, "singlepart_layer")

    # 获取小面积图斑
    small_parcels = []
    with arcpy.da.SearchCursor(temp_singlepart, ["OID@", dlmc_field, dz_field, area_field, "SHAPE@"]) as cursor:
        for row in cursor:
            if row[2] in ['03', '04'] and row[3] < area_threshold:
                small_parcels.append(row)

    print(f"找到 {len(small_parcels)} 个小于 {area_threshold} 平方米的图斑")

    # 处理小面积图斑
    merged_count = 0
    with arcpy.da.Editor(arcpy.env.workspace) as editor:
        for small_parcel in small_parcels:
            oid, land_type, dz, area, shape = small_parcel
            
            # 选择相邻图斑
            arcpy.SelectLayerByLocation_management("singlepart_layer", "BOUNDARY_TOUCHES", shape)
            
            # 在选择集中查找最大的相同地类和DZ的图斑
            max_area = 0
            max_oid = None
            with arcpy.da.SearchCursor("singlepart_layer", ["OID@", dlmc_field, dz_field, area_field]) as neighbor_cursor:
                for neighbor in neighbor_cursor:
                    if neighbor[0] != oid and neighbor[1] == land_type and neighbor[2] == dz and neighbor[3] > max_area:
                        max_area = neighbor[3]
                        max_oid = neighbor[0]
            
            if max_oid:
                try:
                    # 合并几何
                    where_clause = f"OBJECTID IN ({oid}, {max_oid})"
                    with arcpy.da.UpdateCursor(temp_singlepart, ["OID@", "SHAPE@", dlmc_field, dz_field, gridcode_field], where_clause) as update_cursor:
                        shapes = []
                        gridcode = None
                        for update_row in update_cursor:
                            shapes.append(update_row[1])
                            if update_row[0] == max_oid:
                                gridcode = update_row[4]
                            if update_row[0] == oid:
                                update_cursor.deleteRow()
                        
                        if len(shapes) == 2:
                            merged_shape = shapes[0].union(shapes[1])
                            update_cursor.reset()
                            for update_row in update_cursor:
                                if update_row[0] == max_oid:
                                    update_cursor.updateRow([max_oid, merged_shape, land_type, dz, gridcode])
                                    merged_count += 1
                                    break
                except Exception as e:
                    print(f"处理 OID {oid} 时出错: {str(e)}")
                    print(traceback.format_exc())
            
            # 清除选择
            arcpy.SelectLayerByAttribute_management("singlepart_layer", "CLEAR_SELECTION")

            if merged_count % 100 == 0 and merged_count > 0:
                print(f"已处理 {merged_count} 个小面积图斑")

    # 步骤6：更新A要素类
    print("正在更新A要素类...")
    arcpy.CopyFeatures_management(input_A, output_fc)
    
    # 删除那些在原A中DZ为03或04的区域
    arcpy.MakeFeatureLayer_management(output_fc, "output_layer")
    arcpy.SelectLayerByAttribute_management("output_layer", "NEW_SELECTION", f"{dz_field} IN ('03', '04')")
    arcpy.DeleteFeatures_management("output_layer")

    # 将处理后的结果追加到更新后的A
    print("追加处理后的结果...")
    arcpy.Append_management(temp_singlepart, output_fc, "NO_TEST")

    # 计算处理后的要素数
    final_count = int(arcpy.GetCount_management(output_fc)[0])
    print(f"处理完成。更新后的A要素类（包含A的所有字段和B的gridcode字段，小面积已处理）保存为：{output_fc}")
    print(f"最终要素数：{final_count}")

    # 清理临时数据
    print("清理临时数据...")
    for temp_fc in [temp_selected_A, temp_intersect, temp_singlepart]:
        if arcpy.Exists(temp_fc):
            arcpy.Delete_management(temp_fc)

except arcpy.ExecuteError:
    print("ArcPy错误:")
    print(arcpy.GetMessages(2))
except Exception as e:
    print(f"发生错误: {str(e)}")
    print(traceback.format_exc())

In [19]:
# 水田地类和坡位

# 输入要素类
input_A = "DY_SD_CLASS_INTERSECT_reclass_slope_single"
input_B = "reclass_slope"  # 注意：这里使用C作为B

# 临时和输出要素类
temp_selected_A = "in_memory\\Selected_A"
temp_small_A = "in_memory\\Small_A"
temp_large_A = "in_memory\\Large_A"
temp_intersect_small = "in_memory\\Intersect_Small"
temp_intersect_large = "in_memory\\Intersect_Large"
temp_merged = "in_memory\\Merged_Result"
output_final = "Final_Output_FeatureClass"

# 字段名称
dlmc_field = "DLMC"
area_field = "Area_Calc"  # 更改为不太可能冲突的名称
original_area_field = "Orig_Area"
area_ratio_field = "Area_Ratio"
b_attribute_field = "gridcode"  # 更改为不太可能冲突的名称

# 面积阈值
area_threshold = 200

def field_exists(table, field_name):
    return field_name in [f.name for f in arcpy.ListFields(table)]

def add_field_if_not_exists(table, field_name, field_type):
    if not field_exists(table, field_name):
        arcpy.AddField_management(table, field_name, field_type)
        print(f"字段 {field_name} 已添加")
    else:
        print(f"字段 {field_name} 已存在，跳过添加")

try:
    # 步骤1：选择A中DLMC为水田的要素
    print("正在选择A中DLMC为水田的要素...")
    arcpy.MakeFeatureLayer_management(input_A, "A_layer")
    arcpy.SelectLayerByAttribute_management("A_layer", "NEW_SELECTION", f"{dlmc_field} = '水田'")
    arcpy.CopyFeatures_management("A_layer", temp_selected_A)

    # 添加并计算原始面积字段
    add_field_if_not_exists(temp_selected_A, original_area_field, "DOUBLE")
    arcpy.CalculateField_management(temp_selected_A, original_area_field, "!shape.area!", "PYTHON3")

    # 步骤2：分离小面积和大面积地块
    arcpy.MakeFeatureLayer_management(temp_selected_A, "selected_A_layer")
    arcpy.SelectLayerByAttribute_management("selected_A_layer", "NEW_SELECTION", f"{original_area_field} <= {area_threshold}")
    arcpy.CopyFeatures_management("selected_A_layer", temp_small_A)
    arcpy.SelectLayerByAttribute_management("selected_A_layer", "SWITCH_SELECTION")
    arcpy.CopyFeatures_management("selected_A_layer", temp_large_A)

    # 步骤3：对所有地块进行相交操作
    print("正在进行相交操作...")
    arcpy.Intersect_analysis([temp_small_A, input_B], temp_intersect_small, "ALL")
    arcpy.Intersect_analysis([temp_large_A, input_B], temp_intersect_large, "ALL")

    # 步骤4：处理小面积地块
    print("处理小面积地块...")
    add_field_if_not_exists(temp_small_A, b_attribute_field, "TEXT")

    small_dict = {}
    with arcpy.da.SearchCursor(temp_intersect_small, ["OID@", original_area_field, "Shape_Area", "gridcode"]) as cursor:
        for row in cursor:
            oid, orig_area, intersect_area, b_attr = row
            if oid not in small_dict or intersect_area > small_dict[oid][0]:
                small_dict[oid] = (intersect_area, b_attr)

    with arcpy.da.UpdateCursor(temp_small_A, ["OID@", b_attribute_field]) as cursor:
        for row in cursor:
            oid = row[0]
            if oid in small_dict:
                row[1] = small_dict[oid][1]
                cursor.updateRow(row)

    # 步骤5：处理大面积地块
    print("处理大面积地块...")
    add_field_if_not_exists(temp_intersect_large, area_field, "DOUBLE")
    add_field_if_not_exists(temp_intersect_large, area_ratio_field, "DOUBLE")
    arcpy.CalculateField_management(temp_intersect_large, area_field, "!shape.area!", "PYTHON3")
    arcpy.CalculateField_management(temp_intersect_large, area_ratio_field, f"!{area_field}! / !{original_area_field}!", "PYTHON3")

    arcpy.MakeFeatureLayer_management(temp_intersect_large, "intersect_large_layer")
    arcpy.SelectLayerByAttribute_management("intersect_large_layer", "NEW_SELECTION", 
                                            f"{area_ratio_field} < 0.1 AND {area_field} < {area_threshold}")

    if int(arcpy.GetCount_management("intersect_large_layer")[0]) > 0:
        arcpy.Eliminate_management("intersect_large_layer", temp_merged, "LENGTH")
        arcpy.SelectLayerByAttribute_management("intersect_large_layer", "SWITCH_SELECTION")
        arcpy.Append_management("intersect_large_layer", temp_merged, "NO_TEST")
    else:
        arcpy.CopyFeatures_management(temp_intersect_large, temp_merged)

    # 步骤6：合并结果
    arcpy.Append_management(temp_small_A, temp_merged, "NO_TEST")

    # 最后，转换为单部件要素
    arcpy.MultipartToSinglepart_management(temp_merged, output_final)

    print(f"处理完成。最终结果保存为：{output_final}")

    # 清理临时数据
    for temp_fc in [temp_selected_A, temp_small_A, temp_large_A, temp_intersect_small, temp_intersect_large, temp_merged]:
        if arcpy.Exists(temp_fc):
            arcpy.Delete_management(temp_fc)

except arcpy.ExecuteError:
    print("ArcPy错误:")
    print(arcpy.GetMessages(2))
except Exception as e:
    print(f"发生错误: {str(e)}")
    import traceback
    print(traceback.format_exc())

正在选择A中DLMC为水田的要素...
字段 Orig_Area 已添加
正在进行相交操作...
处理小面积地块...
字段 gridcode 已存在，跳过添加
处理大面积地块...
字段 Area_Calc 已添加
字段 Area_Ratio 已添加
处理完成。最终结果保存为：Final_Output_FeatureClass
